In [2]:
#check for nulls
df.isnull().sum()

#get unique values of a dataset
bikedf_dummy['birth year'].unique()


#replace wrong data
#option1
df['Age'] = [np.nan if age=='?' else float(age) for age in df['Age'].values]
#option2
bikedf_dummy['birth year'].replace(to_replace='\\N',value = 0,inplace=True)


#transform a column type from object to int
bikedf_dummy['birth year'] = bikedf_dummy['birth year'].astype(int)



#select df subset ( filter)
Y = bikedf.loc[bikedf["gender"] == 2, "tripduration"]


# 1 - Rename "count" to "riders"
# 2 - Drop "casual" and "registered"
# 3 - Drop only rows where weather == 4
# 4 - Cast "datetime" to actually be a datetime
bike.rename(columns={"count": "riders"}, inplace=True) # solution 1
bike.drop(columns=["casual", "registered"], inplace=True) # solution 2
bike = bike[bike["weather"] != 4] # solution 3
bike["datetime"] = pd.to_datetime(bike["datetime"]) # solution 4


#replace missing values THE MANUAL WAY
#replace missing data using mean
mean_age = income_missing['age'].mean()
income_missing['age_mean_imputed'] = income_missing['age'].fillna(mean_age)
#then check the variance between before and after !
#replace missing data using meadian
median_age = income_missing['age'].median()
income_missing['age_mean_imputed'] = income_missing['age'].fillna(median)
#replace missing data using mode ( most frequent )
mode_age = income_missing['age'].mode()
income_missing['age_mean_imputed'] = income_missing['age'].fillna(mode)


#replace missing values THE SKLEARN WAY
from sklearn.impute import SimpleImputer


#Charts

#heatmap
plt.figure(figsize=(12,12))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm');


#heatmap>|0.5| only
pearson_corr_features = df.corr()

corr_data = np.tril(pearson_corr_features,k=-1) # only looking at bottom heatmap triangle
corr_data = abs(corr_data)>0.50 # only visualizing notable correlations 

# plotting
fig = px.imshow(corr_data, x=pearson_corr_features.columns, y=pearson_corr_features.index,
                color_continuous_scale=px.colors.qualitative.Plotly, width=800, height=800,
                title="Heat map: Feature correlations with >0.5 correlation coefficient")
fig.update(layout_coloraxis_showscale=False)
fig.show()

NameError: name 'df' is not defined